# SQLITE Database Analysis

Requires parts database to already be downloaded and unzipped ```jlcpcb-components.sqlite3```

[Setup Python Virtual Environment in VSCode](/venv-setup.md)

In [ ]:
import sqlite3

conn = sqlite3.connect(r"jlcpcb-components.sqlite3")
cur = conn.cursor()
freelist_count = cur.execute("PRAGMA freelist_count").fetchone()[0]
page_size = cur.execute("PRAGMA page_size").fetchone()[0]
page_count = cur.execute("PRAGMA page_count").fetchone()[0]
conn.close()

print(f"Unused Pages: {freelist_count}, Used Pages: {page_count}, Page size: {page_size} Bytes")

Unused Pages: 0, Used Pages: 263010, Page size: 4096 Bytes
